In [1]:
#DATA WRANGLING
import pandas as pd # Dataframes
from pandas.io.json import json_normalize # JSON wrangler
import statsapi # Python wrapper MLB data API

In [2]:
#DATA STORAGE
#from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

In [3]:
#DATA MANIPULATION AND MODELLING
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn_pandas import DataFrameMapper, FunctionTransformer, gen_features, pipeline
from sklearn_pandas.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import xgboost as xgb
import os

In [4]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [17]:
schedule = statsapi.schedule(start_date="03/28/2018", end_date="05/01/2018", team=136)
full = json_normalize(schedule)
gamepks= full['game_id']

In [20]:
gamepks_2018 = list(gamepks.unique())
len(gamepks_2018)

29

In [21]:
# Get one game from API
list_for_new_df = []
#gamepks = [566389]
for game in gamepks_2018:
    #print(game)
    curr_game = statsapi.get('game_playByPlay',{'gamePk':game})

    ### 3. Extract play-by-play data and store into dataframe.

    # Only care about the allPlays key 
    curr_plays = curr_game.get('allPlays')

    # Coerce all plays into a df
    curr_plays_df = json_normalize(curr_plays)

    ###################################
    # Build target table
    ###################################


    # Data from allPlays
    ap_sel_cols = ['about.atBatIndex', 'matchup.batSide.code', 'matchup.pitchHand.code', 'count.balls'
              ,'count.strikes', 'count.outs']

    # Data from playEvents
    plev_sel_cols = ['details.type.code', 'details.type.description', 
            'details.call.code', 'details.call.description', 
            'details.isBall', 'isPitch', 'details.isStrike'
            ,'pitchData.breaks.breakAngle'
            ,'pitchData.breaks.breakLength', 'pitchData.breaks.breakY'
            ,'pitchData.breaks.spinDirection', 'pitchData.breaks.spinRate'
            ,'pitchData.coordinates.aX'
            , 'pitchData.coordinates.aY','pitchData.coordinates.aZ', 'pitchData.coordinates.pX'
            , 'pitchData.coordinates.pZ', 'pitchData.coordinates.pfxX', 'pitchData.coordinates.pfxZ'
            , 'pitchData.coordinates.vX0', 'pitchData.coordinates.vY0', 'pitchData.coordinates.vZ0'
            , 'pitchData.coordinates.x', 'pitchData.coordinates.x0', 'pitchData.coordinates.y'
            , 'pitchData.coordinates.y0','pitchData.coordinates.z0', 'pitchData.endSpeed'
            , 'pitchData.startSpeed', 'pitchNumber', 'pitchData.zone'
           ]

    # Now go through each row. If there is nested list, json_normalize it
    #for index, row in test_df.head(2).iterrows(): #Just using first 2 rows for testing
    for index, row in curr_plays_df.iterrows(): #Just using first 2 rows for testing

        # saw playEvents is a nested list, so json_normalize it
        play_events_df = json_normalize(row['playEvents'])

        #     # look at runners
        #     runners_df = json_normalize(row['runners'])

        # Loop through THIS NESTED dataframe and NOW build the row for the new df    
        for plev_ind, plev_row in play_events_df.iterrows():

            # Instantiate new dict, which will be a single row in target df
            curr_dict = {}
            curr_dict['game_pk'] = game

            # Loop through each list, adding their respective values to curr_dict
            for col_ap in ap_sel_cols:
                if col_ap in curr_plays_df.columns:
                    curr_dict[col_ap] = row[col_ap]
                else:
                    curr_dict[col_ap] = np.nan
                #print(row['about.atBatIndex'])

            for col_plev in plev_sel_cols:
                if col_plev in play_events_df.columns:
                    curr_dict[col_plev] = plev_row[col_plev]
                else:
                    curr_dict[col_plev] = np.nan

            # collect row dictionary into list
            list_for_new_df.append(curr_dict)

In [24]:
len(list_for_new_df)

9067

In [25]:
pitches_df = pd.DataFrame(list_for_new_df)

In [27]:
pitches_df.head(10)

,about.atBatIndex,count.balls,count.outs,count.strikes,details.call.code,details.call.description,details.isBall,details.isStrike,details.type.code,details.type.description,...,pitchData.coordinates.vZ0,pitchData.coordinates.x,pitchData.coordinates.x0,pitchData.coordinates.y,pitchData.coordinates.y0,pitchData.coordinates.z0,pitchData.endSpeed,pitchData.startSpeed,pitchData.zone,pitchNumber
0,0,1,1,0,B,Ball - Called,True,False,FF,Four-Seam Fastball,...,-9.54,146.72,-2.46,216.58,50.0,5.64,82.3,89.6,13.0,1.0
1,0,1,1,0,X,Hit Into Play - Out(s),False,False,SI,Sinker,...,-7.10,117.07,-2.07,200.69,50.0,5.80,83.1,90.6,13.0,2.0
2,1,1,2,0,B,Ball - Called,True,False,SI,Sinker,...,-4.91,156.29,-2.20,174.75,50.0,5.90,83.6,91.4,13.0,1.0
3,1,1,2,0,X,Hit Into Play - Out(s),False,False,SI,Sinker,...,-6.28,132.08,-2.14,198.39,50.0,5.72,82.8,90.9,7.0,2.0
4,2,3,3,2,S,Strike - Swinging,False,True,SI,Sinker,...,-5.99,113.29,-2.07,189.93,50.0,5.63,83.1,90.9,8.0,1.0
5,2,3,3,2,S,Strike - Swinging,False,True,CU,Curveball,...,-1.63,146.76,-2.29,200.34,50.0,5.95,73.4,80.1,13.0,2.0
6,2,3,3,2,B,Ball - Called,True,False,FF,Four-Seam Fastball,...,-5.02,188.63,-2.37,158.59,50.0,5.92,84.3,92.4,11.0,3.0
7,2,3,3,2,B,Ball - Called,True,False,SI,Sinker,...,-7.05,133.81,-2.10,229.01,50.0,5.64,80.3,87.2,13.0,4.0
8,2,3,3,2,B,Ball - Called,True,False,CU,Curveball,...,-3.54,119.28,-2.34,221.90,50.0,5.81,74.4,80.9,13.0,5.0
9,2,3,3,2,S,Strike - Swinging,False,True,CU,Curveball,...,-1.29,100.55,-2.20,197.26,50.0,5.83,74.6,81.1,9.0,6.0
